In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"



In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
num_epochs = 50
batch_size = 40
learning_rate = 0.001

cuda


In [3]:
transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize( 
       (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010) 
    )
])
train_dataset = torchvision.datasets.CIFAR100(
    root= './data', train = True,
    download =True, transform = transform)
test_dataset = torchvision.datasets.CIFAR100(
    root= './data', train = False,
    download =True, transform = transform)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
train_loader = torch.utils.data.DataLoader(train_dataset
    , batch_size = batch_size
    , shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset
    , batch_size = batch_size
    , shuffle = True)
n_total_step = len(train_loader)
print(n_total_step)


1250


In [5]:
model = models.vgg16(pretrained = False)
input_lastLayer = model.classifier[6].in_features
model.classifier[6] = nn.Linear(input_lastLayer,100)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum=0.9,weight_decay=5e-4)


In [6]:
import datetime
import time
now = time.localtime()
now = "%04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
now = now.replace(":", "_").replace("/", "_")
f = open("results/" +now + ".txt", "w")

In [ ]:


for epoch in range(num_epochs):
    n_corrects = 0
    n_labels = 0
    for i, (imgs , labels) in enumerate(train_loader):
        imgs = imgs.to(device)
        labels = labels.to(device)

        labels_hat = model(imgs)
        loss_value = criterion(labels_hat, labels)
        loss_value.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        n_labels += labels.size(0)
        n_corrects += (labels_hat.argmax(axis=1)==labels).sum().item()
        
        if (i+1) % 250 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, step: {i+1}/{n_total_step}: loss = {loss_value:.5f}, acc = {100*(n_corrects/n_labels):.2f}%')
            f.write(f'epoch {epoch+1}/{num_epochs}, step: {i+1}/{n_total_step}: loss = {loss_value:.5f}, acc = {100*(n_corrects/n_labels):.2f}%\n')
            
    with torch.no_grad():
        number_corrects = 0
        number_samples = 0
        for i, (test_images_set , test_labels_set) in enumerate(test_loader):
            test_images_set = test_images_set.to(device)
            test_labels_set = test_labels_set.to(device)

            y_predicted = model(test_images_set)
            labels_predicted = y_predicted.argmax(axis = 1)
            number_corrects += (labels_predicted==test_labels_set).sum().item()
            number_samples += test_labels_set.size(0)
        print(f'Overall accuracy {(number_corrects / number_samples)*100}%')
        print()
        f.write(f'Overall accuracy {(number_corrects / number_samples)*100}%\n\n')
f.close()

epoch 1/50, step: 250/1250: loss = 4.44984, acc = 1.11%
epoch 1/50, step: 500/1250: loss = 4.59315, acc = 1.11%
epoch 1/50, step: 750/1250: loss = 4.42967, acc = 1.36%
epoch 1/50, step: 1000/1250: loss = 4.49999, acc = 1.61%
epoch 1/50, step: 1250/1250: loss = 4.46509, acc = 1.79%
Overall accuracy 3.0%

epoch 2/50, step: 250/1250: loss = 4.35819, acc = 2.65%
epoch 2/50, step: 500/1250: loss = 4.46376, acc = 2.90%
epoch 2/50, step: 750/1250: loss = 4.48530, acc = 3.12%
epoch 2/50, step: 1000/1250: loss = 4.37226, acc = 3.35%
epoch 2/50, step: 1250/1250: loss = 4.24650, acc = 3.58%
Overall accuracy 4.2299999999999995%

epoch 3/50, step: 250/1250: loss = 4.28838, acc = 5.08%
epoch 3/50, step: 500/1250: loss = 4.26785, acc = 4.95%
epoch 3/50, step: 750/1250: loss = 4.23497, acc = 5.17%
epoch 3/50, step: 1000/1250: loss = 3.97182, acc = 5.45%
epoch 3/50, step: 1250/1250: loss = 3.64908, acc = 5.98%
Overall accuracy 10.13%

epoch 4/50, step: 250/1250: loss = 4.20569, acc = 10.51%
epoch 4/50,